In [0]:

# Create a download link for the 'data.zip' (zipped folder in e2e_project)
# and use 'wget' to directly save the data file into Colab's virtual machine.
!wget --header="Host: doc-0c-6s-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" --header="Cookie: AUTH_7hmq5qi5820iffhdqtvl1fasq7mq1eha=13366280707070436374|1530561600000|97r2pdkih8lrft4lld4ik6ultq746h6v" --header="Connection: keep-alive" "https://doc-0c-6s-docs.googleusercontent.com/docs/securesc/lftjl6vk3idvd1g2urce9ircg4cv3q5d/oo4ogahq1mk8inu06orl846d59r6d2o3/1530561600000/13366280707070436374/13366280707070436374/1sm_aZu9eMGB0WdCtP678iYYlwGWGsFP2?e=download" -O "data.zip" -c
!unzip data.zip
!ls

In [0]:
# Execute this section to Set up your Google Drive with Colab. This will help you in saving the model directly to your Google Drive
# Reference: https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q#scrollTo=c99EvWo1s9-x

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls drive/


In [4]:
# Install the packages

import numpy as np
import pandas as pd
import skimage
import skimage.transform as sktransform
from sklearn import model_selection
import random
import matplotlib.image as mpimg
import os
import shutil
import sys



# https://keras.io/
!pip install -q keras
from keras.callbacks import Callback
import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.models import load_model
import math

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))


print("Keras Version: ",keras.__version__)
print("Tensorflow Version: ",tf.__version__)

Using TensorFlow backend.


Keras Version:  2.1.6
Tensorflow Version:  1.9.0-rc1


In [0]:
# Data Augmentation, Weight Callbacks

# Cameras we will use
cameras = ['left', 'center', 'right']
cameras_steering_correction = [.25, 0., -.25]

def preprocess(image, top_offset=.375, bottom_offset=.125):
    """
    Applies preprocessing pipeline to an image: crops `top_offset` and `bottom_offset`
    portions of image, resizes to 32x128 px and scales pixel values to [0, 1].
    """
    top = int(top_offset * image.shape[0])
    bottom = int(bottom_offset * image.shape[0])
    image = sktransform.resize(image[top:-bottom, :], (32, 128, 3))
    return image

def generate_samples(data,root_path, bs=128,augment=True):
    """
    Keras generator yielding batches of training/validation data.
    Applies data augmentation pipeline if `augment` is True.
    """
    
    #import pdb
    #pdb.set_trace()
    while True:
        # Generate random batch of indices
        indices = np.random.permutation(data.count()[0])
        batch_size = bs #128

        for batch in range(0, len(indices), batch_size):
            batch_indices = indices[batch:(batch + batch_size)]
            # Output arrays
            x = np.empty([0, 32, 128, 3], dtype=np.float32)
            y = np.empty([0], dtype=np.float32)
            # Read in and preprocess a batch of images

            for i in batch_indices:
                # Randomly select camera
                camera = np.random.randint(len(cameras)) if augment else 1
                # Read frame image and work out steering angle
                image = mpimg.imread(os.path.join(root_path, data[cameras[camera]].values[i].strip()))
                image.setflags(write=1)
                #print('Flags: ',image.flags)
                angle = data.steering.values[i] + cameras_steering_correction[camera]
                if augment:
                    # Add random shadow as a vertical slice of image
                    h, w = image.shape[0], image.shape[1]
                    [x1, x2] = np.random.choice(w, 2, replace=False)
                    k = h / (x2 - x1)
                    b = - k * x1
                    #print('Flags: ',image.flags)
                    for i in range(h):
                        c = int((i - b) / k)
                        image[i, :c, :] = (image[i, :c, :] * .5).astype(np.int32)
                        
                # Randomly shift up and down while preprocessing
                v_delta = .05 if augment else 0
                image = preprocess(
                    image,
                    top_offset=random.uniform(.375 - v_delta, .375 + v_delta),
                    bottom_offset=random.uniform(.125 - v_delta, .125 + v_delta)
                )
                # Append to batch
                x = np.append(x, [image], axis=0)
                y = np.append(y, [angle])
                
            # Randomly flip half of images in the batch. Uncomment the following if you want to include flipping.
            
            #flip_indices = random.sample(range(x.shape[0]), int(x.shape[0] / 2))
            #x[flip_indices] = x[flip_indices, :, ::-1, :]
            #y[flip_indices] = -y[flip_indices]
            yield (x, y)
            

 

In [0]:
#Keeps track of model weights by saving them at the end of each epoch.

class WeightsLogger(Callback):
    def __init__(self, root_path):
      super(WeightsLogger, self).__init__()
      self.weights_root_path = os.path.join(root_path, 'weights/')
      shutil.rmtree(self.weights_root_path, ignore_errors=True)
      os.makedirs(self.weights_root_path, exist_ok=True)

    def on_epoch_end(self, epoch, logs={}):
        self.model.save_weights(os.path.join(self.weights_root_path, 'model_epoch_{}.h5'.format(epoch + 1)))

In [7]:
# Split Training and Validation data

local_project_path = ''
local_data_path = os.path.join(local_project_path, 'data')

sys.path.append(local_project_path) 


if __name__ == '__main__':
    # Read the data
    df = pd.io.parsers.read_csv(os.path.join(local_data_path, 'driving_log.csv'))
    # Split data into training and validation sets
    df_train, df_valid = model_selection.train_test_split(df, test_size=.1)

print("train: ",df_train.shape[0])
print("validation: ",df_valid.shape[0])

train:  6219
validation:  691


In [9]:
# Network Building blocks

def add_denseblock(input, growth_rate = 12, numLayers=12, dropout_rate = 0.2):
    temp = input
    for _ in range(numLayers):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        conv1xC = Conv2D(growth_rate,(1,3), use_bias=False ,padding='same',dilation_rate=2)(relu)
        conv1xCx1 = Conv2D(growth_rate,(3,1), use_bias=False ,padding='same',dilation_rate=2)(conv1xC)
        concat = Concatenate(axis=-1)([temp,conv1xCx1])
        temp = concat
        
    return temp
  
def add_transition(input, growth = 16, dropout_rate = 0.2):
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(growth, (1,1), use_bias=False ,padding='same',dilation_rate=2)(relu)
    if dropout_rate>0:
        Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg


def output_layer(input):
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)

    output = Dense(1,activation='tanh')(flat)
    
    return output



print("Network building blocks are ready")

Network building blocks are ready


In [14]:
#Define the hyperparameters : growth rate in densenet
growth_rate_net1 = 14
numLayers_net1 = 7

growth_rate_net2 = 10
numLayers_net2 = 11

dropout_rate = 0.0
input_shape=(32, 128, 3)

input = Input(shape=input_shape)
First_Conv2D = Conv2D(growth_rate_net1, (3,3), use_bias=False ,padding='same',dilation_rate=2)(input)

First_Block = add_denseblock(First_Conv2D, growth_rate_net1, numLayers_net1, dropout_rate)
First_Transition = add_transition(First_Block, growth_rate_net1, dropout_rate)

Second_Block = add_denseblock(First_Transition, growth_rate_net2, numLayers_net2, dropout_rate) 
Second_Transition = add_transition(Second_Block, growth_rate_net2, dropout_rate)

output = output_layer(Second_Transition)

model = Model(inputs=[input], outputs=[output])

model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 128, 3)   0                                            
__________________________________________________________________________________________________
conv2d_114 (Conv2D)             (None, 32, 128, 14)  378         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_63 (BatchNo (None, 32, 128, 14)  56          conv2d_114[0][0]                 
__________________________________________________________________________________________________
activation_63 (Activation)      (None, 32, 128, 14)  0           batch_normalization_63[0][0]     
__________________________________________________________________________________________________
conv2d_115

In [15]:

# Learning Rate Schedule

def step_decay(epoch):
    initial_lrate = 0.005
    drop = 0.1
    epochs_drop = 25.0
    lrate = initial_lrate * math.pow(drop,  
    math.floor((1+epoch)/epochs_drop))
    return lrate
  
lrate = LearningRateScheduler(step_decay)  

# Determine Loss function and Optimizer

adm = keras.optimizers.Adam(lr = 0.0001,beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='mean_squared_error',optimizer=adm)
print("Model Compilation Successful")

# Training Training

epoch = 50
batch_size = 128

history = model.fit_generator(
    generate_samples(df_train,local_data_path, batch_size),
    steps_per_epoch=df_train.shape[0]/batch_size,
    epochs=epoch,
    callbacks=[lrate,WeightsLogger(root_path=local_project_path)],
    validation_data=generate_samples(df_valid, local_data_path, augment=False),
    validation_steps = df_valid.shape[0]/batch_size,
    initial_epoch=0, verbose=1)

# Directly save the model to your google drive
model.save('drive/model.h5')
with open('drive/model.json', 'w') as file:
    file.write(model.to_json())

Model Compilation Successful
Epoch 1/50


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


49/48 [==============================] - 82s 2s/step - loss: 0.1514 - val_loss: 0.1600
Epoch 2/50
49/48 [==============================] - 58s 1s/step - loss: 0.0826 - val_loss: 0.0852
Epoch 3/50
49/48 [==============================] - 61s 1s/step - loss: 0.0815 - val_loss: 0.1081
Epoch 4/50
49/48 [==============================] - 61s 1s/step - loss: 0.0768 - val_loss: 0.1022
Epoch 5/50
49/48 [==============================] - 61s 1s/step - loss: 0.0732 - val_loss: 0.0745
Epoch 6/50
49/48 [==============================] - 60s 1s/step - loss: 0.0723 - val_loss: 0.0684
Epoch 7/50
10/48 [=====>........................] - ETA: 38s - loss: 0.064849/48 [==============================] - 60s 1s/step - loss: 0.0705 - val_loss: 0.0594
Epoch 8/50
49/48 [==============================] - 61s 1s/step - loss: 0.0696 - val_loss: 0.0799
Epoch 9/50
49/48 [==============================] - 60s 1s/step - loss: 0.0711 - val_loss: 0.0868
Epoch 10/50
49/48 [==============================] - 62s 1s/step 